In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import shutil

from scipy import signal

import sys
sys.path.append('.../bats-code')

import bat_functions as bf
from CountLine import CountLine
import cv2
import matplotlib as mpl
import utm
import matplotlib.pyplot as plt


import rasterio

In [ ]:
plot_folder = '.../bats-data/plots'
save_folder = os.path.join(plot_folder, 'camera-map')
os.makedirs(save_folder, exist_ok=True)

In [ ]:
shift = 0
HFOV = 85.8 # degrees
HCONST = 1454.9 # pixels
WCONST = 1453.7 # pixels
FRAME_WIDTH = 2704 - (2 * shift)
WINGSPAN = .8 # meters, max extent while flying 

In [ ]:
camera_locations = {'FibweParking2': [-12.5903393, 30.2525047],	
                    'FibweParking': [-12.5903393, 30.2525047],
                    'Chyniangale': [-12.5851284, 30.245529],	
                    'BBC': [-12.5863538, 30.2484985],
                    'Sunset': [-12.585784, 30.240003],
                    'NotChyniangale': [-12.5849206,	30.2436135],
                    'MusoleParking': [-12.58787, 30.2401],	
                    'MusolePath2': [-12.589544,	30.242488],	
                    'MusolePath': [-12.589544,	30.242488],
                    'Puku': [-12.584838, 30.24137],	
                    'FibwePublic': [-12.592537, 30.2515924],	
                    'MusoleTower': [-12.589434, 30.244736],
                    }
all_camera_utms = bf.latlong_dict_to_utm(camera_locations)

In [ ]:
center_utm = {'middle': np.array([200450, 8606950]),
              'right': np.array([200800, 8606900])}

In [ ]:
root_folder = ".../kasanka-bats/processed/deep-learning"
observations_root = os.path.join(root_folder, "observations")
observations = {}
day_folders = sorted(glob.glob(os.path.join(observations_root, '*')))
for day_folder in day_folders:
    obs_files = sorted(glob.glob(os.path.join(day_folder, '*.npy')))
    date = os.path.basename(day_folder)
    observations[date] = {}
    for obs_file in obs_files:
        camera = os.path.splitext(obs_file)[0].split('-')[-1]
        obs = np.load(obs_file, allow_pickle=True)
        # .item() to get dict from inside the array that was wrapped around
        # it when using np.save()
        observations[date][camera] = obs.item()

In [ ]:
# Remove observations to exclude (because camera ran out of batteries etc.)
exclude=True
# Manually exclude cameras that had issues
observations['17Nov']['MusoleParking']['exclude'] = True
observations['18Nov']['MusolePath']['exclude'] = True
observations['20Nov']['MusolePath']['exclude'] = True
if exclude:
    good_obs = {}
    for date, day_obs in observations.items():
        good_obs[date] = {}
        for camera, obs in day_obs.items():
            if 'exclude' in obs.keys():
                if obs['exclude']:
                    continue
            good_obs[date][camera] = obs
    observations = good_obs

In [ ]:
map_file = '.../bats-data/maps/kasanka-utm.tiff'
forest_map_dataset = rasterio.open(map_file)

forest_map = [forest_map_dataset.read(band_ind) for band_ind in range(1, 4)]
forest_map = np.array(forest_map)
forest_map = forest_map.transpose(1, 2, 0)

plt.imshow(forest_map)

map_left = forest_map_dataset.bounds.left
map_top = forest_map_dataset.bounds.top
width = np.abs(map_left - forest_map_dataset.bounds.right).astype(int)
height = np.abs(map_top - forest_map_dataset.bounds.bottom).astype(int)

x_shift = .1
y_shift = .1
legend_length = 100 #meters

legend_line_x = [map_left + x_shift*width, 
               map_left + legend_length + x_shift*width]
legend_line_y = [map_top - y_shift*height, 
                 map_top - y_shift*height]
line_x = []
line_y = []
for x, y in zip(legend_line_x, legend_line_y):
    legend_plot = forest_map_dataset.index(x, y)
    line_x.append(legend_plot[1])
    line_y.append(legend_plot[0])

plt.plot(line_x, line_y, c='w', linewidth=1)
print(line_x, line_y)

In [ ]:
def get_camera_width_meters(height, field_of_view):
    """ Return width of view in meters height away from camera.
    
    height: height in meters
    field_of_view: degrees"""
    fov_radians = field_of_view * np.pi / 180
    width = 2 * height * np.tan(fov_radians/2)
    return width

In [ ]:
def get_camera_borders(camera_utms, center_utm, jitter=False):
    """ Get angles around forest center that evenly bisect camera positions.
    
    camera_utms: dict pairs of camera names and location info
    center_utm: 2d np.array, location of forest center
    jitter: if True, don't actually bisect cameras at midpoint but drawn
        from a gaussian
    """
    
    camera_border = {}
    camera_angles = bf.get_camera_angles(camera_utms, center_utm)
    for camera, camera_utm in camera_utms.items():
        min_neg = -10000
        min_pos = 100000
        # for border case where focal is positive angle 
        # and closest cclock is negative
        max_pos = 0 
        # for same case a last comment
        all_pos = True 
        # for border case where focal is positive angle 
        # and closest cclock is negative
        max_neg = 0 
        # for same case a last comment
        all_neg = True 
        max_camera = None
        camera_border[camera] = {'cclock': None,
                                 'cclock_angle': None,
                                 'clock': None,
                                 'clock_angle': None
                                }
        for alt_camera, alt_camera_utm in camera_utms.items():
            
            if camera == alt_camera:
                continue

            dif = camera_angles[camera] - camera_angles[alt_camera]
            if dif < 0:
                all_pos = False
                if dif > min_neg:
                    min_neg = dif
                    camera_border[camera]['cclock'] = alt_camera
                    camera_border[camera]['cclock_angle'] = dif / 2
                if dif < max_neg:
                    max_neg = dif 
                    max_camera = alt_camera

            if dif > 0:
                all_neg = False
                if dif < min_pos:
                    min_pos = dif
                    camera_border[camera]['clock'] = alt_camera
                    camera_border[camera]['clock_angle'] = dif / 2
                if dif > max_pos:
                    max_pos = dif 
                    max_camera = alt_camera

        if all_pos:
            camera_border[camera]['cclock'] = max_camera
            camera_border[camera]['cclock_angle'] = (max_pos - 2*np.pi) / 2
        if all_neg:
            camera_border[camera]['clock'] = max_camera
            camera_border[camera]['clock_angle'] = (max_neg + 2*np.pi) / 2
            
    if jitter:
        for camera, border_info in camera_border.items():
            camera_angle = camera_angles[camera]
            clockwise_camera = border_info['clock']
            angle_dif = border_info['clock_angle']
            # Three sttandard deviations is between camera pair
            jitter_angle = np.random.normal(scale=angle_dif/3)
            jitter_angle = np.maximum(-border_info['clock_angle'], 
                                      jitter_angle)
            jitter_angle = np.minimum(border_info['clock_angle'],
                                      jitter_angle)

            camera_border[camera]['clock_angle'] += jitter_angle
            if camera_border[camera]['clock_angle'] < 0:
                camera_border[camera]['clock_angle'] += (2 * np.pi)
            if camera_border[camera]['clock_angle'] >= (2 * np.pi):
                camera_border[camera]['clock_angle'] -= (2 * np.pi)
            camera_border[clockwise_camera]['cclock_angle'] += jitter_angle
            if camera_border[clockwise_camera]['cclock_angle'] < -2 * np.pi:
                camera_border[clockwise_camera]['cclock_angle'] += (2 * np.pi)
            if camera_border[clockwise_camera]['cclock_angle'] >= (2 * np.pi):
                camera_border[clockwise_camera]['cclock_angle'] -= (2 * np.pi)
        
            
    return camera_border


In [ ]:
center = 'middle'
date = '16Nov'
fontsize = 20
camera_fontsize = 15

should_save = True

os.makedirs(save_folder, exist_ok=True)

def rotate(vec, angle):
    new_x = vec[0] * np.cos(angle) - vec[1] * np.sin(angle)
    new_y = vec[0] * np.sin(angle) + vec[1] * np.cos(angle)
    return np.array([new_x, new_y])

import matplotlib.cm as cm

demo_height = 45
effective_camera_width = get_camera_width_meters(demo_height, HFOV)
print('eff camera width:', effective_camera_width)

for center in ['middle']:
    color = 'r'
    total_angle = 0
    
    fig, ax = plt.subplots(figsize=(20,20))
    ax.imshow(forest_map)

    camera_utms = bf.get_camera_locations(
        observations[date], all_camera_utms, exclude=True)
    camera_border = get_camera_borders(camera_utms, 
                                          center_utm[center],
                                          jitter=False)
    camera_angles = bf.get_camera_angles(camera_utms, 
                                         center_utm[center])
    for camera, camera_utm in camera_utms.items():
    #     c = cm.viridis((camera_angles[camera] + np.pi)/ (2*np.pi))
        camera_plot = forest_map_dataset.index(*camera_utm)
        camera_plot = [camera_plot[1], camera_plot[0]]
        ax.scatter(*camera_plot, c='w', s=100)
        cclock = camera_border[camera]['cclock']
        clock = camera_border[camera]['clock']
        ax.annotate(camera, camera_plot, fontsize=camera_fontsize, 
                    xytext=[camera_plot[0], camera_plot[1]-60], 
                    ha='center', va='top', color='white', 
                    bbox=dict(boxstyle = "square",
                              facecolor = "gray", 
                              edgecolor="gray",
                              alpha=.5)
                   )

        mid = rotate(np.array(camera_utms[camera])-np.array(center_utm[center]), 
                     -camera_border[camera]['clock_angle'])
        mid = mid + np.array(center_utm[center])
        mid_plot = forest_map_dataset.index(*mid)
        mid_plot = [mid_plot[1], mid_plot[0]]
        center_plot = forest_map_dataset.index(*center_utm[center])
        center_plot = [center_plot[1], center_plot[0]]
        ax.plot([mid_plot[0], center_plot[0]], [mid_plot[1], center_plot[1]], c=color)

        mid = rotate(np.array(camera_utms[camera])-np.array(center_utm[center]), 
                     -camera_border[camera]['cclock_angle'])
        mid = mid + np.array(center_utm[center])
        mid_plot = forest_map_dataset.index(*mid)
        mid_plot = [mid_plot[1], mid_plot[0]]
        ax.plot([mid_plot[0], center_plot[0]], 
                [mid_plot[1], center_plot[1]], c=color)
        
        # Add camera extent info:
        camera_angle = camera_angles[camera]
        dy = (effective_camera_width / 2) * np.cos(camera_angle)
        dx = (effective_camera_width / 2) * np.sin(camera_angle)
        extent_1 = camera_utm + np.array([dx, -dy])
        extent_2 = camera_utm + np.array([-dx, dy])

        ex1_plot = forest_map_dataset.index(*extent_1)
        ex1_plot = [ex1_plot[1], ex1_plot[0]]

        ex2_plot = forest_map_dataset.index(*extent_2)
        ex2_plot = [ex2_plot[1], ex2_plot[0]]

        ax.plot([ex1_plot[0], ex2_plot[0]], 
                 [ex1_plot[1], ex2_plot[1]],
                c='w', linewidth=2)
    
        
    #     ax.scatter(*(mid + np.array(center_utm)), c='r', s=50)
        ax.set_aspect('equal', adjustable='box')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    #     ax.set_xlabel('Meters (UTM)', fontsize=15)
    #     ax.set_ylabel('Meters (UTM)', fontsize=15)

        total_angle += -camera_border[camera]['cclock_angle']
        total_angle += camera_border[camera]['clock_angle']

    print('total_angle', total_angle * 180 / np.pi)
    ax.scatter(*center_plot, s=100, c='r')
    
    # Add legend
    map_left = forest_map_dataset.bounds.left
    map_top = forest_map_dataset.bounds.top
    width = np.abs(map_left - forest_map_dataset.bounds.right).astype(int)
    height = np.abs(map_top - forest_map_dataset.bounds.bottom).astype(int)

    x_shift = .72
    y_shift = .95
    legend_length = 500 #meters

    legend_line_x = [map_left + x_shift*width, 
                   map_left + legend_length + x_shift*width]
    legend_line_y = [map_top - y_shift*height, 
                     map_top - y_shift*height]
    line_x = []
    line_y = []
    for x, y in zip(legend_line_x, legend_line_y):
        legend_plot = forest_map_dataset.index(x, y)
        line_x.append(legend_plot[1])
        line_y.append(legend_plot[0])

    ax.plot(line_x, line_y, c='w', linewidth=2)
#     ax.annotate(f'{legend_length} meters', 
#                 (np.mean(line_x), line_y[0]), 
#                 fontsize=fontsize, 
#                 xytext=(np.mean(line_x), line_y[0]-25), 
#                 ha='center', va='top', color='white', 
# #                 bbox=dict(boxstyle = "square",
# #                           facecolor = "gray", alpha=.7)
#                )
        
    ax.set_aspect('equal', adjustable='box')
    title = 'camera-weighting-center-{}'.format(center)
#     ax.set_title(title.replace('-', ' '), size=30)
    if should_save:
        fig.savefig(os.path.join(save_folder, title + '.png'), bbox_inches='tight')

In [ ]:
os.path.join(save_folder, title + '.png')

### multiple forest centers

In [ ]:
forest_border = [[-12.585957, 30.242762],
                 [-12.586763, 30.246229],
#                  [-12.589854, 30.250597],
#                  [-12.591381, 30.249095],
                 [-12.589182, 30.245566],
                 [-12.587557, 30.241598]
                ]

In [ ]:
map_file = '.../bats-data/maps/kasanka-utm.tiff'
forest_map_dataset = rasterio.open(map_file)
forest_map = [forest_map_dataset.read(band_ind) for band_ind in range(1, 4)]
forest_map = np.array(forest_map)
forest_map = forest_map.transpose(1, 2, 0)
width = np.abs(forest_map_dataset.bounds.left - forest_map_dataset.bounds.right).astype(int)
height = np.abs(forest_map_dataset.bounds.top - forest_map_dataset.bounds.bottom).astype(int)
area = np.zeros((height, width), dtype=np.uint8)

In [ ]:
camera_utm_array = []
for cam_utm in all_camera_utms.values():
    camera_utm_array.append(cam_utm)
camera_utm_array = np.array(camera_utm_array)

forest_utms = []
for f_latlon in forest_border:
    f_utm = utm.from_latlon(*f_latlon)
    forest_utms.append([f_utm[0], f_utm[1]])
forest_utms = np.array(forest_utms)

norm_forest = np.copy(forest_utms)
area_x_origin = forest_map_dataset.bounds.left
area_y_origin = forest_map_dataset.bounds.bottom
norm_forest[:, 0] = forest_utms[:, 0] - area_x_origin
norm_forest[:, 1] = forest_utms[:, 1] - area_y_origin
area = cv2.drawContours(area, 
                        [norm_forest.astype(np.int32)], 
                        -1, 255, -1)

In [ ]:
plt.imshow(area)

In [ ]:
plt.imshow(forest_map)
for point in forest_utms:
    row, col = forest_map_dataset.index(point[0], point[1])
    print(point[1], point[0], row, col)
    plt.scatter(col, row)

In [ ]:
def get_day_total(observations, center_utm, all_camera_utms, 
                  frame_width, wingspan, exclude=False, 
                  correct_darkness=False, parameters=None):
    """ Estimate total number of bats based on all observation counts
    and corespoinding camera locations.
    
    observations: dict of all observations for a specific day
    center_utm: estimated location of forest center
    all_camera_utms: dict of the utm locations of each camera
    frame_width: width of camera frame in pixels
    wingspan: estimated wingspan off all bats in meters
    exlude: to manually remove certain cameras, ie shut off early etc.
    correct_darkness: divide by accuracy estimated for given darkness
    parameters: param values of linear piecewise function for darkness
        error correction. Required if correct_darkness is True
    """
    
    frac_sum = 0
    total = 0
    obs_totals = []
    
    camera_utms = bf.get_camera_locations(observations, all_camera_utms, exclude=True)
    camera_fractions = bf.get_camera_fractions(camera_utms, center_utm)
    camera_distances = bf.get_camera_distances(camera_utms, center_utm)
    for obs in observations.values():
        if exclude:
            if 'exclude' in obs.keys():
                if obs['exclude']:
                    print('excluding...')
                    continue
        
        
        obs['multiplier'] = bf.combined_bat_multiplier(frame_width, 
                                                    wingspan, 
                                                    obs['mean_wing'], 
                                                    camera_distances[obs['camera']]
                                                   )
        if correct_darkness:
            assert parameters is not None, "Must pass parameters if correcting for darkness."
            acc = piecewise_linear(obs['darkness'], *parameters)
            obs['total_darkness'] = np.sum(obs['multiplier'] * obs['direction'] * (1/acc))
        obs['total'] = np.sum(obs['multiplier'] * obs['direction'])
        obs['total_unscaled'] = np.sum(obs['direction'])
        obs['fraction_total'] = camera_fractions[obs['camera']]
        frac_sum += obs['fraction_total']
        if correct_darkness:
            total += obs['total_darkness'] * obs['fraction_total']
            obs_totals.append(obs['total_darkness'])
        else:
            total += obs['total'] * obs['fraction_total']
            obs_totals.append(obs['total'])

    if len(obs_totals) > 0:
        mean_total = np.mean(obs_totals)
    else:
        mean_total = 0
    return total, mean_total

In [ ]:
exclude = True
scale = 10
done=False
darkness_parameters = [1.57454778e+01, 9.37398964e-01, 7.18914388e-02, -1.27575036e-04]
total_mean_maps = []
total_weighted_maps = []
for _ in range(len(observations)):
#     total_mean_maps.append(np.zeros(((int(area.shape[0]/scale)+1, int(area.shape[1]/scale)+1))))
    total_weighted_maps.append(np.zeros(((int(area.shape[0]/scale)+1, int(area.shape[1]/scale)+1))))                  
for x_ind, x in enumerate(range(0, area.shape[1], scale)):
    for y_ind, y in enumerate(range(0, area.shape[0], scale)):
        if area[y, x] == 0:
            continue
        x_utm = x + area_x_origin
        y_utm = y + area_y_origin
        center_utm = np.array([x_utm, y_utm])
        for day_ind, day in enumerate(observations.values()):
            day_total, day_total_mean = get_day_total(day, center_utm, 
                                                         all_camera_utms, 
                                                         FRAME_WIDTH, WINGSPAN, 
                                                         exclude=exclude,
                                                         correct_darkness=False,
                                                         parameters=darkness_parameters)
            
#             total_mean_maps[day_ind][y_ind, x_ind] = day_total_mean
            total_weighted_maps[day_ind][y_ind, x_ind] = day_total
            done=True
#         if done:
#             break
#     if done:
#         break


                         
             
# for map_ind, total_map in enumerate(total_mean_maps):
#     total_mean_maps[map_ind] = cv2.resize(total_map, (area.shape[1], area.shape[0]), 
#                                           interpolation=cv2.INTER_NEAREST)
for map_ind, total_map in enumerate(total_weighted_maps):
    total_weighted_maps[map_ind] = cv2.resize(total_map, (area.shape[1], area.shape[0]), 
                                          interpolation=cv2.INTER_NEAREST)  

In [ ]:
day_total

In [ ]:
plt.imshow(masked)
plt.colorbar()

In [ ]:
vals = np.ravel(total_weighted_maps[0])
vals = vals[vals>1000]
plt.hist(vals)

In [ ]:
area_mask = np.where(area, True, False)
for weighted, (date, day_obs) in zip(total_weighted_maps, observations.items()):
    weighted = weighted 
    mean = np.mean(weighted)
    plt.figure()
    plt.imshow(weighted-mean)

    masked = np.ma.masked_where(weighted<0, 
                                weighted)
    print(masked.max(), np.mean(weighted))

In [ ]:
max_count = -10000000
min_count = 10000000000
for weighted_map in total_weighted_maps[:]:
    weighted_map = weighted_map 
    weighted_map = np.ravel(weighted_map)
    weighted_map = weighted_map[weighted_map>0]
    mean = np.mean(weighted_map)
    
    weighted_map = (weighted_map-mean) / mean
    day_max = np.max(weighted_map)
    day_min = np.min(weighted_map)
    if day_max > max_count:
        max_count = day_max
    if day_min < min_count:
        min_count = day_min
print(max_count, min_count)

In [ ]:
np.min(weighted_map)

In [ ]:
import matplotlib.cm as cm
import matplotlib

alpha=1.0

should_save = True

# save_folder = os.path.join(plot_folder, 'various-centers')
# os.makedirs(save_folder, exist_ok=True)

cmap = cm.seismic
for weighted, (date, day_obs) in zip(total_weighted_maps, observations.items()):
    fig, axs = plt.subplots(1, 1, figsize=(20,7))
    camera_utms = bf.get_camera_locations(day_obs, all_camera_utms, exclude=exclude)
    print(camera_utms.keys())
    weighted = weighted 
    mean = np.ravel(weighted)
    mean = np.mean(mean[mean>0])
    for camera, cam_utm in camera_utms.items():
        cv2.circle(weighted, 
                   (int(cam_utm[0] - area_x_origin), 
                    int(cam_utm[1] - area_y_origin)), 
                   15, mean,
                 -1)

    axs.imshow(forest_map)
    
    weighted = (weighted-mean) / mean
    weighted = cv2.resize(weighted, (forest_map.shape[1], forest_map.shape[0]))
    alpha_mask = np.where(weighted>-1, 1.0, 0.0) * alpha
    n = axs.imshow(weighted[::-1], cmap=cmap, 
                   vmin=min_count, vmax=max_count, 
                   alpha=alpha_mask[::-1])
    axs.set_aspect('equal', adjustable='box')
    axs.set_xticks([]) 
    axs.set_yticks([]) 
#     axs.set_title('Weighted Cameras', fontsize=15)

#     norm = matplotlib.colors.Normalize(vmin=min_count, vmax=max_count)
    norm = matplotlib.colors.Normalize(vmin=-max_count, vmax=max_count)
    sm = cm.ScalarMappable(norm=norm, cmap=cmap)
    cbar = fig.colorbar(sm)
    cbar.set_label('Fractional difference from mean', rotation=270, labelpad=15, fontsize=15)
    cbar.ax.tick_params(labelsize=15)
#     l, y=1.05, rotation=0
    title = 'total-bats-from-various-centers-map-{}'.format(date)
    fig.suptitle(date, size=30)
    if should_save:
        fig.savefig(os.path.join(save_folder, title + '.png'), bbox_inches='tight')